In [3]:
import cv2
from datetime import datetime
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage.color import label2rgb
from skimage.measure import regionprops_table
from skimage.segmentation import clear_border
from scipy import ndimage

import omero.clients
from getpass import getpass
from omero.gateway import BlitzGateway, rint, rdouble, rstring
from omero.model import RoiI, MaskI, ImageI, PolygonI, LengthI
from omero.model.enums import UnitsLength
import struct

import requests
import json
csvtocsvw_url='https://csvtocsvw.matolab.org'

def post_request(url, headers, data, files=None):
    try:
        if files:
            # should crate a multipart form upload
            response = requests.post(url, data=data, headers=headers, files=files)
        else:
            # a application json post request
            response = requests.post(url, data=json.dumps(data), headers=headers)
        response.raise_for_status()
        
    except requests.exceptions.RequestException as e:
        #placeholder for save file / clean-up
        raise SystemExit(e) from None
    return response

def annotate_csv(csv_url: str, encoding: str='auto',):
    ## curl -X 'POST' \ 'https://csvtocsvw.matolab.org/api/annotation' \ -H 'accept: application/json' \ -H 'Content-Type: application/json' \ -d '{ "data_url": "https://github.com/Mat-O-Lab/CSVToCSVW/raw/main/examples/example.csv", "separator": "auto", "header_separator": "auto", "encoding": "auto" }'
    url = csvtocsvw_url+"/api/annotate"
    data = { 
        "data_url": csv_url,
        "encoding": encoding
    }
    headers = {'Content-type': 'application/json', 'Accept': 'application/json'}
    r = post_request(url, headers, data).json()
    metafilename=filename.rsplit('.')[0]+'-metadata.json'
    file=json.dumps(r['filedata']).encode('utf-8')
    print('csvw annotation file created, suggested name: {}'.format(metafilename))
    return metafilename, file

# need credentials
username=input("Username: ")
password=getpass("OMERO Password: ")

meta_extractor_api="https://metadata.omero.matolab.org/api/"

HOST = 'wss://wss.omero.matolab.org'
client = omero.client(HOST)

def test_connect():
    session = client.createSession(username, password)
    with BlitzGateway(client_obj=client) as conn:
        print("Connected as {}".format(conn.getUser().getName()))
        print("User ID: {}".format(conn.getUser().getId()))
        print("User Full Name: {}".format(conn.getUser().getFullName()))

        print("Your Groups:")
        for g in conn.getGroupsMemberOf():
            print("   Name:", g.getName(), " ID:", g.getId())
        group = conn.getGroupFromContext()
        print("Current group: ", group.getName())

def get_datasets():
    session = client.createSession(username, password)
    with BlitzGateway(client_obj=client) as conn:
        #demo project
        project=conn.getObject("Project", 51)
        return list(project.listChildren())

# Load images in a specified dataset method
def get_images(dataset_id):
    """
    Load the images in the specified dataset
    :param conn: The BlitzGateway
    :param dataset_id: The dataset's id
    :return: The Images or None
    """
    session = client.createSession(username, password)
    with BlitzGateway(client_obj=client) as conn:
        dataset = conn.getObject("Dataset", dataset_id)
        images = []
        if dataset:
            for image in dataset.listChildren():
                images.append(image)
            if len(images) == 0:
                return None

        for image in images:
            print("---- Loaded image ID:", image.id)
        
        # Print dataset ID and name
        print("Dataset ID:", dataset.getId())
        print("Dataset Name:", dataset.getName())
    return images

def create_mask(mask_bytes, bytes_per_pixel=1):
    if bytes_per_pixel == 2:
        divider = 16.0
        format_string = "H"  # Unsigned short
        byte_factor = 0.5
    elif bytes_per_pixel == 1:
        divider = 8.0
        format_string = "B"  # Unsigned char
        byte_factor = 1
    else:
        message = "Format %s not supported"
        raise ValueError(message)
    steps = math.ceil(len(mask_bytes) / divider)
    mask = []
    for i in range(int(steps)):
        binary = mask_bytes[
            i * int(divider):i * int(divider) + int(divider)]
        format = str(int(byte_factor * len(binary))) + format_string
        binary = struct.unpack(format, binary)
        s = ""
        for bit in binary:
            s += str(bit)
        mask.append(int(s, 2))
    return bytearray(mask)

# We have a helper function for creating an ROI and linking it to new shapes
def create_roi(img_id, shapes):
    # create an ROI, link it to Image
    roi = RoiI()
    # use the omero.model.ImageI that underlies the 'image' wrapper
    session = client.createSession(username, password)
    with BlitzGateway(client_obj=client) as conn:
        img = conn.getObject("Image", img_id)
        roi.setImage(ImageI(img_id, False))
        for shape in shapes:
            roi.addShape(shape)
        # Save the ROI (saves any linked shapes too)
        updateService=conn.getUpdateService()
        return updateService.saveAndReturnObject(roi)

def delete_rois(image_id):
    session = client.createSession(username, password)
    with BlitzGateway(client_obj=client) as conn:
        roi_service = conn.getRoiService()
        result = roi_service.findByImage(image_id, None)
        roi_ids = [roi.id.val for roi in result.rois]
        if roi_ids:
            conn.deleteObjects("Roi", roi_ids)
        print("deleted rois: {}".format(roi_ids)) 

def rgba_to_int(red, green, blue, alpha=255):
    """ Return the color as an Integer in RGBA encoding """
    return int.from_bytes([red, green, blue, alpha],
                      byteorder='big', signed=True)

def print_image_details(img_id):
    # Retrieve information about the image
    session = client.createSession(username, password)
    with BlitzGateway(client_obj=client) as conn:
        image = conn.getObject("Image", img_id)
        print("Image Name:", image.getName())
        print("Image Name:", image.id)
        print("Image Description:", image.getDescription())
        print("Image SizeX:", image.getSizeX())
        print("Image SizeY:", image.getSizeY())
        print("Image SizeZ:", image.getSizeZ())
        print("Image SizeC:", image.getSizeC())
        print("Image SizeT:", image.getSizeT())

        x = image.getName()

        # List Channels (loads the Rendering settings to get channel colors)
        for channel in image.getChannels():
            print('Channel:', channel.getLabel())
            print('Color:', channel.getColor().getRGB())
            print('Lookup table:', channel.getLut())
            print('Is reverse intensity?', channel.isReverseIntensity())
            

        print(image.countImportedImageFiles())
        file_count = image.countFilesetFiles()
        # list files
        if file_count > 0:
            for orig_file in image.getImportedImageFiles():
                name = orig_file.getName()
                path = orig_file.getPath()
                print(name)
                print(path)
    return x
def get_grayscale(img_id):
    session = client.createSession(username, password)
    with BlitzGateway(client_obj=client) as conn:
        image = conn.getObject("Image", img_id)
        z = image.getSizeZ() / 2
        t = 0
        rendered_image = image.renderImage(z, t)
        image_array = np.asarray(rendered_image)
        # Convert the image to grayscale
        gray = cv2.cvtColor(image_array, cv2.COLOR_RGB2GRAY)
    return gray
    
# def create_omero_roi_masks(image_id):
#     mask_x = 50
#     mask_y = 50
#     mask_h = 100
#     mask_w = 100
#     # Create [0, 1] mask
#     mask_array = np.fromfunction(
#         lambda x, y: (x * y) % 2, (mask_w, mask_h))
#     # Set correct number of bytes per value
#     mask_array = mask_array.astype(np.uint8)
#     # Convert the mask to bytes
#     mask_array = mask_array.tobytes()
#     # Pack the bytes to a bit mask
#     mask_packed = create_mask(mask_array, 1)

#     # Define mask's fill color
#     mask_color = ColorHolder()
#     mask_color.setRed(255)
#     mask_color.setBlue(0)
#     mask_color.setGreen(0)
#     mask_color.setAlpha(100)
#     # create an ROI with a single mask
#     mask = MaskI()
#     mask.setTheC(rint(0))
#     mask.setTheZ(rint(0))
#     mask.setTheT(rint(0))
#     mask.setX(rdouble(mask_x))
#     mask.setY(rdouble(mask_y))
#     mask.setWidth(rdouble(mask_w))
#     mask.setHeight(rdouble(mask_h))
#     mask.setFillColor(rint(mask_color.getInt()))
#     mask.setTextValue(rstring("test-Mask"))
#     mask.setBytes(mask_packed)
#     return create_roi(image_id, [mask])

def create_omero_roi_polygons(image_id,contours):
    the_t = 0
    the_z = 0
    polygons = []
    # create an ROI with a single polygon
    for i,contour in enumerate(contours):
        polygon = PolygonI()
        polygon.theZ = rint(the_z)
        polygon.theT = rint(the_t)
        color = list(np.random.choice(range(256), size=3))
        polygon.strokeColor = rint(rgba_to_int(color[0], color[1], color[2]))
        polygon.fillColor = rint(rgba_to_int(color[0], color[1], color[2]))
        
        polygon.strokeWidth = LengthI(1, UnitsLength.PIXEL)
        pts=["{},{}".format(point[:,0][0],point[:,1][0]) for point  in contour]
        pts_list=" ".join(pts)
        polygon.points = rstring(pts_list)
        #print(pts_list[:30])
        polygon.textValue = rstring("Precipitate"+str(i))
        polygons.append(polygon)
    #print(polygon.info())
    #print(polygon.__dir__())
    create_roi(image_id, polygons)
    print("added {} polygon shapes to image".format(len(polygons)))
    
test_connect()
datasets=get_datasets()

DNSException: exception ::Ice::DNSException
{
    error = 11001
    host = wss.omero.matolab.org
}

In [6]:
def run_detection(
        image,
        threshold_method: str="Otsu",
        size_thresh: float= 80,
        dilate_kernel_size: int=3,
        median_filter_radius: int=4,
        save_to_file=False,
        plot=False
        ):

    name=image.name
    print("Image Name: "+name)

    gray=get_grayscale(image.id)

    print("Grayscale image shape:", gray.shape)
    print("Grayscale image data type:", gray.dtype)

    # Apply median filter using OpenCV
    
    #selem = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2*median_filter_radius + 1, 2*median_filter_radius+1))
    filtered = cv2.medianBlur(gray, 2 * median_filter_radius + 1)

    # Display the original and filtered images side by side
    if plot:
        fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(10, 5))
        ax0.imshow(gray, cmap='gray')
        ax0.set_title('Step1: Load Original Image')
        ax0.axis('off')
        ax1.imshow(filtered, cmap='gray')
        ax1.set_title('Step2: Median Blur Filter')
        ax1.axis('off')

    # Apply Otsu's thresholding using OpenCV
    if threshold_method=="Otsu":
        ret, thresh = cv2.threshold(filtered, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        print('Threshold value is {}'.format(ret))
    else:
        print("only Otsu thresholding supported, skippping thresholding")
        thresh=gray

    # Dilate the thresholded image using a 3x3 kernel
    kernel = np.ones((dilate_kernel_size,dilate_kernel_size), np.uint8)
    dilated = cv2.dilate(thresh, kernel)

    if plot:
        fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(10, 5))
        ax0.imshow(thresh, cmap='gray')
        ax0.set_title('Step3: Otsu Threshold')
        ax0.axis('off')
        ax1.imshow(dilated, cmap='gray')
        ax1.set_title('Step4: Dilated Image')
        ax1.axis('off')

    # Remove small objects using OpenCV's morphologyEx function
    morphed = cv2.morphologyEx(dilated, cv2.MORPH_OPEN, kernel, iterations=1)
    contours, hierarchy = cv2.findContours(morphed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area < size_thresh:
            cv2.drawContours(morphed, [cnt], 0, 0, -1)

    # delete all previously defined shapes
    delete_rois(image.id)

    # upload shapes to omero
    create_omero_roi_polygons(image.id,contours)

    # Apply clear border to the dilated image
    mask = morphed  == 255
    mask = clear_border(mask)

    # Label the mask and count the number of precipitates detected
    s = [[1,1,1],[1,1,1],[1,1,1]]
    labeled_mask, num_labels = ndimage.label(mask, structure=s)
    img2 = label2rgb(labeled_mask, bg_label=0)

    # Display the labeled mask
    if plot:
        fig, ax = plt.subplots(figsize=(6, 6))
        ax.imshow(img2)
        ax.set_title('Step5: Contour Finding')
        ax.axis('off')
        plt.show()
        print(f"Number of precipitates detected: {num_labels}")

    plt.show()
    
    

## The Pipline

In [7]:
data=list()
for dataset in datasets:
    images = get_images(dataset.id)

    for image in images:
        startTime = datetime.now().isoformat()
        info = image.name.split('_')
        state=info[0].split("-")
        if len(state)>1:
            anneal_temp, anneal_time = info[0].split("-")
            anneal_temp = float(anneal_temp.rsplit('C',1)[0])
            anneal_time = float(anneal_time.rsplit('h',1)[0])
        else:
            anneal_temp, anneal_time = 0.0, 0.0
        specimen = info[1].split('Sample',1)[-1]
        pos = info[2].split('Sample',1)[-1]

        threshold_method="Otsu"
        size_thresh=80
        dilate_kernel_size=3
        median_filter_radius=4
        info_dict={
            "SpecimenName": specimen,
            "Aging Temp [°C]": float(anneal_temp),
            "Aging Time [h]": float(anneal_time),
            "Creep Stress [MPa]": 0,
            "Dataset": meta_extractor_api+"dataset/"+str(image.id),
            "Image": meta_extractor_api+"image/"+str(image.id),
            "ROIs": meta_extractor_api+"rois/"+str(image.id),
            "DiskRadiusValue [px]": median_filter_radius,
            "Threshold Method": threshold_method,
            "Dilation Kernel Size [px]": dilate_kernel_size,
            "Date": startTime
        }
        run_detection(image,threshold_method="Otsu",size_thresh=80,dilate_kernel_size=3,median_filter_radius=4)
        data.append(info_dict)
    #     break
    # break
df=pd.DataFrame(data)
df.to_csv('detection_runs.csv')



---- Loaded image ID: 83
---- Loaded image ID: 84
---- Loaded image ID: 85
---- Loaded image ID: 74
---- Loaded image ID: 75
---- Loaded image ID: 76
---- Loaded image ID: 77
---- Loaded image ID: 78
---- Loaded image ID: 79
---- Loaded image ID: 80
---- Loaded image ID: 81
---- Loaded image ID: 82
Dataset ID: 53
Dataset Name: 190C_1000h
Image Name: 190C-1000h_Sample1_Stelle 10 DF 30s.dm3
Grayscale image shape: (2048, 2048)
Grayscale image data type: uint8
Threshold value is 61.0
deleted rois: [421]
added 33 polygon shapes to image
Image Name: 190C-1000h_Sample1_Stelle 11 DF 30s.dm3
Grayscale image shape: (2048, 2048)
Grayscale image data type: uint8
Threshold value is 89.0
deleted rois: [422]
added 98 polygon shapes to image
Image Name: 190C-1000h_Sample1_Stelle 12 DF 30s.dm3
Grayscale image shape: (2048, 2048)
Grayscale image data type: uint8
Threshold value is 90.0
deleted rois: [423]
added 100 polygon shapes to image
Image Name: 190C-1000h_Sample1_Stelle 1 DF 20s.dm3
Grayscale imag

DNSException: exception ::Ice::DNSException
{
    error = 11001
    host = wss.omero.matolab.org
}

In [4]:
# annotate csv with metadata, uses already commited csv on main branch
filename, filedata=annotate_csv("https://github.com/BAMresearch/DF-TEM-PAW/raw/integrate-omero-shapes/detection_runs.csv")
with open("filename", "w") as outfile:
    outfile.write(filedata)

SystemExit: 500 Server Error: Internal Server Error for url: https://csvtocsvw.matolab.org/api/annotation

c:\Anaconda3\envs\opencv_skimage\lib\site-packages\IPython\core\interactiveshell.py:3513: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
# Extract properties of each labeled region
props = regionprops_table(labeled_mask, image, properties=('area',
                                                                    'axis_major_length',
                                                                   'axis_minor_length',
                                                                      'centroid',))

# Create a Pandas dataframe from the properties
df = pd.DataFrame(props)

area_in_nm2 = df['area'] * 0.0225
df['area_in_nm2'] = area_in_nm2

radius_in_nm = np.sqrt(df['area_in_nm2']/math.pi)
df['radius_in_nm'] = radius_in_nm
aspect_ratio = df['axis_major_length']/df['axis_minor_length']
df.insert(3, 'aspect_ratio', aspect_ratio)

#To delete small regions...
df = df[df['area'] > 50]

df.head()

### Creating the Meta-data

In [ ]:
date = pd.Timestamp("today").strftime("%Y-%m-%d")

In [ ]:
header = {'BitSizeValue': radius , 
          'DiskRadiusValue[pixel]': 8,
          'OtsuThresholdValue[np]': 'Otsu',
          'KernelValue':3,
          'MaskSetValue':255, 
          'BG_LabelValue':0,
          'Date':date}

In [ ]:
df_header= pd.DataFrame([header])

In [ ]:
df_header.head(4)

### Joining the meta-data to the dataframe

In [ ]:
df_join = df.join(df_header, rsuffix='_right')
df_join.head(5)

In [ ]:
# filling the NaN values
df_join=df_join.fillna(method='ffill')
df_join.tail(5)

### Creating informative columns 

In [ ]:
df_join['Precipitate#'] = range(1,len(df_join)+1)
df_join.head(5)

In [ ]:

desc = {'MaterialState':x[0] , 
         'Sample': x[1],
          'Image': x[2],
         }

df_desc= pd.DataFrame([desc])
df_desc.head()

In [ ]:
#df_join = df.join(df_desc, lsuffix='_MaterialState')
frames = [df_desc, df_join]

result = pd.concat(frames, axis="columns")
result.head(5)

In [ ]:
# Fill the missing data
df_join=result.fillna(method='ffill')
df_join.tail(5)

In [ ]:
df_join['Image'] = df_join['Image'].str.replace('.dm3', '', regex=True)

In [ ]:
def change_column_order(df, col_name, index):
    cols = df.columns.tolist()
    cols.remove(col_name)
    cols.insert(index, col_name)
    return df[cols]

In [ ]:
df_final_result = change_column_order(df_join, 'Precipitate#', 3)

In [ ]:
df_final_result = df_final_result.rename(columns={"centroid-0": "centroid-X", "centroid-1": "centroid-Y"})

In [ ]:
df_final_result.head(5)

In [ ]:
df_final_result.to_csv('190c_2500h_1iteration.csv',  header=False, mode='a',index=False, date_format='%s')